In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 200)

In [2]:
df = pd.read_pickle('FT-data-DSpace/data_processed.pickle')
df.head(1)

,ID,Date,Meeting Number,Agenda Name,Agenda Number,File Type,File Number,File Step,Speaker Name,Speaker Role,Speaker Start,Speaker End,Speech
0,CBAEA326-B201-41AB-A756-029BA1F58D82,2009-10-30T10:00:00,9. møde,[None],0,Formandens meddelelser,,,None,formand,2009-10-30T10:00:18.063,2009-10-30T10:01:04.997,"Mødet er åbnet.I dag er der følgende anmeldelser: Thor Pedersen (V), Mogens Lykketoft (S), Søren Espersen (DF), Holger K. Nielsen (SF) og Helge Adam Møller (KF):Lovforslag nr. L 53 (Forslag til lo..."


In [3]:
df = df[['Date', 'Meeting Number', 'Agenda Number', 'Speech']]
df = df.dropna()
df.sample(2)

,Date,Meeting Number,Agenda Number,Speech
31608,2010-04-19T10:00:00,77. møde,5,"Tak for det. Hr. Jesper Langballe har jo sådan set redegjort for en del af den meget alvorlige problemstilling, som vi står over for, og vi har jo også før i Folketingssalen haft diskussioner. I f..."
38939,2010-06-10T10:00:00,105. møde,4,Så er det ordføreren.


In [4]:
df_aggregated = df.groupby(by =['Date', 'Meeting Number', 'Agenda Number'])['Speech'].apply(lambda x: ',,'.join(x)) \
                  .reset_index()
df_aggregated.sample(2)

,Date,Meeting Number,Agenda Number,Speech
434,2009-12-17T10:00:00,34. møde,13,"Der er ikke stillet ændringsforslag.Der er ingen, der beder om ordet, og så går vi til afstemning.,,Der kan stemmes … (munterhed). Ja, ja, det, der bliver misforstået, er, at man i beslutningsfors..."
1786,2010-10-26T13:00:00,7. møde,4,"Forhandlingen er åbnet. Hr. Karsten Nonbo som ordfører for Venstre.,,Det lovforslag, vi skal behandle nu, har to elementer. Det første element er en udvidelse af strafferammen for groft uagtsomt h..."


In [5]:
#### TOKENIZATION AND STEMMING
import nltk
from nltk.stem.snowball import DanishStemmer

stopwords = nltk.corpus.stopwords.words('danish') 
tokenizer = nltk.RegexpTokenizer(r"\w+")
stemmer = DanishStemmer()

In [6]:
# tokenize the string and remove all punktuations
df_aggregated['Unstemmed'] = df_aggregated['Speech'].apply(lambda x: tokenizer.tokenize(x)) 

In [7]:
df_aggregated.sample(3)

,Date,Meeting Number,Agenda Number,Speech,Unstemmed
10154,2016-11-11T10:00:00,15. møde,0,Mødet er åbnet.I dag er der følgende anmeldelser:Forsvarsministeren (Peter Christensen):Lovforslag nr. L 80 (Forslag til lov om jobordning for veteraner).Udenrigsministeren (Kristian Jensen):Lovfo...,"[Mødet, er, åbnet, I, dag, er, der, følgende, anmeldelser, Forsvarsministeren, Peter, Christensen, Lovforslag, nr, L, 80, Forslag, til, lov, om, jobordning, for, veteraner, Udenrigsministeren, Kri..."
2030,2010-12-01T13:00:00,27. møde,8,"Hr. Karl H. Bornhøft.,,Tak. Spørgsmålet lyder: Hvor længe har ministeren tænkt sig at lade den situation fortsætte, hvor manglende adgang til upublicerede data fra lægemiddelforsøg betyder, at læg...","[Hr, Karl, H, Bornhøft, Tak, Spørgsmålet, lyder, Hvor, længe, har, ministeren, tænkt, sig, at, lade, den, situation, fortsætte, hvor, manglende, adgang, til, upublicerede, data, fra, lægemiddelfor..."
5679,2013-05-07T13:00:00,93. møde,6,"Der er ikke stillet ændringsforslag. Ønsker nogen at udtale sig?Det er ikke tilfældet. Vi går til afstemning.,,Der stemmes om lovforslagets endelige vedtagelse nu.Jeg slutter afstemningen.Lovforsl...","[Der, er, ikke, stillet, ændringsforslag, Ønsker, nogen, at, udtale, sig, Det, er, ikke, tilfældet, Vi, går, til, afstemning, Der, stemmes, om, lovforslagets, endelige, vedtagelse, nu, Jeg, slutte..."


In [8]:
### stem the tokens
df_aggregated['Stemmed'] = df_aggregated['Unstemmed'].apply(lambda x: [stemmer.stem(y) for y in x if y.isalnum()])

In [9]:
## remove stop words and numbers
df_aggregated['Clean'] = df_aggregated['Stemmed'].apply(lambda x: [y for y in x if not y.isnumeric() and not y in stopwords])

In [10]:
### data aggregated
df_aggregated.to_pickle('FT-data-DSpace/data_agreggated_with_agendas.pickle')
df_aggregated.sample(2)

,Date,Meeting Number,Agenda Number,Speech,Unstemmed,Stemmed,Clean
5136,2013-01-16T13:00:00,45. møde,3,"Jeg skal bare lige gentage, fordi nye folk kommer ind i salen, at vi har afskaffet oplæsning af spørgsmålene. Vi går direkte til besvarelse fra ministeren. Når den primære spørger har haft de to f...","[Jeg, skal, bare, lige, gentage, fordi, nye, folk, kommer, ind, i, salen, at, vi, har, afskaffet, oplæsning, af, spørgsmålene, Vi, går, direkte, til, besvarelse, fra, ministeren, Når, den, primære...","[jeg, skal, bar, lig, gentag, fordi, nye, folk, kom, ind, i, sal, at, vi, har, afskaf, oplæsning, af, spørgsmål, vi, går, direk, til, besvar, fra, minist, når, den, primær, spørg, har, haft, de, t...","[bar, lig, gentag, fordi, nye, folk, kom, sal, afskaf, oplæsning, spørgsmål, går, direk, besvar, minist, primær, spørg, haft, to, først, rund, kom, medspørg, to, rund, a, minut, tak, formand, spør..."
332,2009-12-08T13:00:00,28. møde,4,"Der er ikke stillet ændringsforslag.Der er ingen, der beder om ordet. Så går vi til afstemning.,,Der stemmes om lovforslagets endelige vedtagelse, og der kan stemmes.Afstemningen slutter.For stemt...","[Der, er, ikke, stillet, ændringsforslag, Der, er, ingen, der, beder, om, ordet, Så, går, vi, til, afstemning, Der, stemmes, om, lovforslagets, endelige, vedtagelse, og, der, kan, stemmes, Afstemn...","[der, er, ikk, stil, ændringsforslag, der, er, ing, der, bed, om, ord, så, går, vi, til, afstemning, der, stem, om, lovforslag, end, vedtag, og, der, kan, stem, afstemning, slut, for, stemt, 114, ...","[ikk, stil, ændringsforslag, ing, bed, ord, så, går, afstemning, stem, lovforslag, vedtag, kan, stem, afstemning, slut, stemt, v, s, df, sf, kf, rv, el, la, imod, stemt, hverk, ell, imod, stemt, l..."
